In [40]:
import numpy as np
import pandas as pd
import glob
import cv2
import ntpath 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import mahotas
import os
import pickle 
from sklearn.decomposition import PCA

In [41]:
def color_moments(pic):
    pic=np.double(pic)
    R=pic[:,:,0]
    G=pic[:,:,1]
    B=pic[:,:,2]
    colorFeature=[np.mean(R[:]),np.std(R[:]),np.mean(G[:]),np.std(G[:]),np.mean(B[:]),np.std(B[:])];
    colorFeature=colorFeature/np.mean(colorFeature)
    return colorFeature

def shapeFeatures(pic):
     # Calculate Moments
     moments = cv2.moments(cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY))
     # Calculate Hu Moments
     huMoments = cv2.HuMoments(moments).flatten()
     shapeFeat=huMoments/np.mean(huMoments);
     return shapeFeat

# feature-descriptor-2: histogram
def fd_histogram(pic):
    # convert the image to HSV color-space
    pic = cv2.cvtColor(pic, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist(pic, [0, 1, 2], None, [8, 2, 2], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

# feature-descriptor-2: Haralick Texture
def fd_haralick(pic):
    # convert the image to grayscale
    gray = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick
 
def getFeatures(pic):
    featuresGlbal=np.zeros((209,30))
    fv_Moments=color_moments(pic)
    fv_histogram  = fd_histogram(pic)
    fv_haralick= fd_haralick(pic)
    fv_hu_moments = shapeFeatures(pic);
    
    #featuresGlbal= np.append(fv_haralick, fv_histogram);
    
    features = np.append(fv_Moments, fv_haralick);
    features = np.append(features, fv_hu_moments);
    featuresGlbal = np.append(features, fv_histogram);
    return featuresGlbal
    
def createFeatures(fold1,fold2):
    dircs1 = os.listdir(fold1)
    dircs2 = os.listdir(fold1)
    
    with open('featuresTrain.csv', 'w') as out:
        for dirc in dircs1:
            for pic in glob.glob(fold1+'\\'+dirc+'\\*.jpg'):
                img = cv2.resize(cv2.imread(pic),None, fx = 0.1, fy = 0.1)
                feats = np.append(dirc,getFeatures(img))
                out.write('%s\n'% ','.join(feats))
                
    with open('featuresValidation.csv', 'w') as out:
        for dirc in dircs2:
            for pic in glob.glob(fold2+'\\'+dirc+'\\*.jpg'):
                img = cv2.resize(cv2.imread(pic),None, fx = 0.1, fy = 0.1)
                feats = np.append(dirc,getFeatures(img))
                out.write('%s\n'% ','.join(feats))

In [42]:
createFeatures('train','validation')
    
data = pd.read_csv('featuresTrain.csv', sep = ',').to_numpy()
x_train = data[:,1:]
y_train = data[:,0]

In [43]:
# normalise
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(x_train)
x_train = scaler.transform(x_train)

In [44]:
# PCA
pca = PCA(.3) 
pca.fit(x_train)
x_train = pca.transform(x_train)
    
#lda = LinearDiscriminantAnalysis(n_components=1)
#x_train = lda.fit(x_train, y_train).transform(x_train)
    


In [48]:
x_train

array([[-6.53293723e-01, -8.62185613e-02],
       [-6.93953286e-01, -1.43461174e-01],
       [ 7.17866877e-01,  3.57487073e-02],
       [-7.02140976e-01,  1.19243846e-01],
       [-7.81138510e-01, -6.48525961e-02],
       [ 8.51768953e-01,  1.11033297e-01],
       [-5.48734705e-01,  8.10630720e-01],
       [-1.03078206e+00,  2.46075238e+00],
       [ 1.09271551e+00, -4.67858830e-03],
       [-6.17845486e-01, -4.24103686e-01],
       [-6.44964726e-01, -3.72512864e-01],
       [-6.22576114e-01, -3.59404730e-01],
       [ 7.44733832e-01, -1.46832679e-01],
       [ 5.14097526e-01,  2.91739641e-01],
       [ 4.60482594e-01,  3.67798469e-01],
       [ 4.92378993e-01,  3.80465798e-01],
       [ 1.41420861e+00,  3.13671456e-02],
       [ 2.95186878e-01,  2.42093301e-01],
       [-2.83134902e-01,  1.09526900e+00],
       [-2.55763329e-01,  1.01652499e+00],
       [ 7.56996078e-01,  4.46581190e-01],
       [ 8.71594206e-01,  4.51710614e-01],
       [-4.52937784e-01,  8.54773389e-01],
       [-6.

In [45]:
#    #Build your classifier KNN
    #classifier = KNeighborsClassifier(n_neighbors=5)
    
# Build your classifier SVM
classifier = SVC(kernel='rbf',gamma=0.4)
    
# Save the trained model as a pickle string. 
saved_model = pickle.dumps(classifier)
    
# Load the pickled model 
classifier_from_pickle = pickle.loads(saved_model)

# fit model
classifier_from_pickle.fit(x_train,y_train)
print('score de training with SVC :',classifier_from_pickle.score(x_train,y_train))

data3 = pd.read_csv('featuresValidation.csv', sep = ',').to_numpy()
x_validation = data3[:,1:]
y_validation = data3[:,0] 

scaler.fit(x_validation)
x_validation = scaler.transform(x_validation)
      


score de training with SVC : 0.6906077348066298


In [47]:
#    pca.fit(x_validation)
#    x_validation = pca.transform(x_validation)
    
x_validation = lda.transform(x_validation)

print('score de validation:',classifier_from_pickle.score(x_validation,y_validation))

x_test = []
y_pred = []
for pic in glob.glob("test\\*.jpg"):
    x_test.append(ntpath.basename(pic))
    feature = getFeatures(cv2.imread(pic))
    # noramlize
    feature = feature.reshape(1,-1)
    scaler.fit(feature)
    feature = scaler.transform(feature)
    # pca
#        pca.fit(feature)
#        feature = pca.transform(feature)

    feature = lda.transform(feature)
    print(classifier_from_pickle.predict(feature))
    y_pred.append('Plante_'+classifier_from_pickle.predict(feature)[0]) 

r= pd.DataFrame({'name':x_test,'classe':y_pred})
r.to_csv("resultat.csv",index=False)

ValueError: X.shape[1] = 1 should be equal to 2, the number of features at training time